In [0]:
import numpy as np
import pickle

from music21 import instrument, note, stream, chord

from tensorflow.keras.models import model_from_json

In [2]:
# Load notes list
with open('notes.pickle', 'rb') as filename:
    notes = pickle.load(filename)
print("Loaded notes list from disk")

# Load x
with open('x.pickle', 'rb') as filename:
    x = pickle.load(filename)
print("Loaded x from disk")

# Load model and weights
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

model.load_weights("model.h5")
print("Loaded model from disk")

Loaded notes list from disk
Loaded x from disk
Loaded model from disk


In [0]:
# Number of unique notes
n_vocab = len(set(notes))

pitchnames = sorted(set(item for item in notes))
# Map notes/chords into integer categories
notes_categories = dict((number, note) for number, note in enumerate(pitchnames))

In [0]:
generate_length = 500
first_index = np.random.randint(0, len(x) - 1)
pattern = x[first_index]

predict_y = []

for i in range(generate_length):
    predict_x = np.reshape(pattern, (1, len(pattern), 1))
    predict_x = predict_x / float(n_vocab)
    prediction = model.predict(predict_x)
    
    # Get highest probability from encoded matrix
    index = np.argmax(prediction)
    result = notes_categories[index]
    predict_y.append(result)
    pattern.append(index)
    
    # Remove first note until starting sequence is all removed
    pattern = pattern[1:]

In [0]:
offset = 0
output_notes = []

for element in predict_y:
    # chord element
    if ('.' in element) or element.isdigit():
        notes_in_chord = element.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # note element
    else:
        new_note = note.Note(element)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    
    offset += 0.5

In [6]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = 'output.mid')

'output.mid'